In [ ]:
# !pip install scanpy
import os
import numpy as np
import pandas as pd
import scanpy as sc
import time
from scipy.stats import pearsonr
#
#more 10x datasets 
#https://support.10xgenomics.com/single-cell-multiome-atac-gex/datasets/


#switch to keras for custom ae
#https://blog.keras.io/building-autoencoders-in-keras.html
#merging nerual networks
#https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



In [ ]:
print(tf.test.gpu_device_name())

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
#make a shortcut to emily's shared drive folder in your drive so you can access the data at
import os
os.listdir('/content/drive/My Drive/CS260C/methyl_impute')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Project Proposal Brainstorming.gdoc',
 'Peek_data.ipynb',
 'adult_cortex_CG-CH_OLDCOPY.h5ad',
 'adult_cortex_CG_CH.h5ad',
 'tf_multiclass_prediction.ipynb',
 'classifier_results.gslides',
 'Presentation.gslides',
 'VAE_impute (variational).ipynb',
 'ch_isImputed.parq',
 'cg_isImputed.parq',
 'PredictionModel.ipynb',
 'VAE_impute (vanilla+tuned).ipynb',
 'VAE_impute (vanilla) (custom dropout).ipynb',
 'VAE_impute (variational+loss).ipynb',
 'Project_Report.gdoc']

In [ ]:
adata=sc.read_h5ad('/content/drive/My Drive/CS260C/methyl_impute/adult_cortex_CG_CH.h5ad')
adata

AnnData object with n_obs × n_vars = 11945 × 44772
    obs: 'sample', 'L1', 'L2', 'L3', 'true_batch', 'age', 'age_groups', 'leiden'
    var: 'batch'
    uns: 'L2_colors', 'L3_colors', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
#collect top marker genes

sc.tl.rank_genes_groups(adata, 'L3',n_genes=20)
# sc.pl.rank_genes_groups(adata, sharey=False)
markers=[]
for i in range(27):
    for j in range(20):
        if adata.uns['rank_genes_groups']['names'][j][i] not in markers:
            markers.append(adata.uns['rank_genes_groups']['names'][j][i])
"""adata2=adata[:,markers]
sc.pp.scale(adata2)
sc.pl.heatmap(adata2, markers, groupby='L3',vmax=1,vmin=-1)"""

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/scanpy/tools/_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/usr/local/lib/python3.7/dist-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()

"adata2=adata[:,markers]\nsc.pp.scale(adata2)\nsc.pl.heatmap(adata2, markers, groupby='L3',vmax=1,vmin=-1)"

In [ ]:
adata=adata[:,markers]
adata

View of AnnData object with n_obs × n_vars = 11945 × 351
    obs: 'sample', 'L1', 'L2', 'L3', 'true_batch', 'age', 'age_groups', 'leiden'
    var: 'batch'
    uns: 'L2_colors', 'L3_colors', 'leiden', 'neighbors', 'pca', 'umap', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [38]:
##### read in matices for ch and cg
ch_impute_mask=pd.read_parquet('/content/drive/My Drive/CS260C/methyl_impute/ch_isImputed.parq')
ch_new_cols = [x+"_CH" for x in ch_impute_mask.columns.values]
ch_impute_mask.columns=ch_new_cols
print(ch_impute_mask.shape)
print(ch_impute_mask.head())

(15732, 32067)
                                 WASH7P_CH  AL627309.1_CH  WASH9P_CH  \
Sample                                                                 
20210111-1863-preAb-PFC-B12_B21       True           True       True   
20210111-1863-preAb-PFC-B12_B16       True           True       True   
20210111-1863-preAb-PFC-B12_A3        True           True       True   
20210111-1863-preAb-PFC-B12_A5        True           True       True   
20210111-1863-preAb-PFC-B12_A14       True           True       True   

                                 AP006222.1_CH  AL669831.3_CH  AL669831.1_CH  \
Sample                                                                         
20210111-1863-preAb-PFC-B12_B21           True           True           True   
20210111-1863-preAb-PFC-B12_B16           True           True           True   
20210111-1863-preAb-PFC-B12_A3            True           True           True   
20210111-1863-preAb-PFC-B12_A5            True          False           True   


In [40]:
cg_impute_mask=pd.read_parquet('/content/drive/My Drive/CS260C/methyl_impute/cg_isImputed.parq')
cg_new_cols = [x+"_CG" for x in cg_impute_mask.columns.values]
cg_impute_mask.columns=cg_new_cols
print(cg_impute_mask.shape)
print(cg_impute_mask.head())

(15732, 12705)
                                 AL669831.3_CG  LINC01128_CG  AL645608.6_CG  \
Sample                                                                        
20210111-1863-preAb-PFC-B12_B21          False          True           True   
20210111-1863-preAb-PFC-B12_B16           True          True          False   
20210111-1863-preAb-PFC-B12_A3            True          True           True   
20210111-1863-preAb-PFC-B12_A5            True          True           True   
20210111-1863-preAb-PFC-B12_A14          False         False           True   

                                 SAMD11_CG  NOC2L_CG  PLEKHN1_CG  ISG15_CG  \
Sample                                                                       
20210111-1863-preAb-PFC-B12_B21       True     False       False      True   
20210111-1863-preAb-PFC-B12_B16      False     False        True      True   
20210111-1863-preAb-PFC-B12_A3       False      True        True     False   
20210111-1863-preAb-PFC-B12_A5        Tru

In [41]:
X_df = adata.to_df()
print(X_df.head())

region                           GRIN2B_CG  CAMK2A_CG  AGPAT4_CG  TANC2_CG  \
20210111-1863-preAb-PFC-B12_A3    0.851852   0.857143   1.000000  0.822222   
20210111-1863-preAb-PFC-B12_A23   0.946237   0.633663   0.843750  0.806727   
20210111-1863-preAb-PFC-B12_B10   0.736842   0.633663   0.771523  0.846154   
20210111-1863-preAb-PFC-B12_A1    0.494253   0.303030   0.740741  0.565217   
20210111-1863-preAb-PFC-B12_B13   0.747621   0.111111   0.923077  0.914286   

region                           MYT1L_CG  ACOT7_CH   LDB2_CG  GABBR2_CH  \
20210111-1863-preAb-PFC-B12_A3   0.770992  0.011933  0.830189   0.016535   
20210111-1863-preAb-PFC-B12_A23  0.841085  0.010582  0.866667   0.005171   
20210111-1863-preAb-PFC-B12_B10  0.800000  0.018336  0.637500   0.009547   
20210111-1863-preAb-PFC-B12_A1   0.684211  0.019830  0.428571   0.009115   
20210111-1863-preAb-PFC-B12_B13  0.564516  0.011259  0.614035   0.017424   

region                           KHDRBS3_CG  MEF2C_CG  ...  TFAP2B_CG  \
2

In [42]:
print(X.shape)

(11945, 351)


In [57]:
ch_col_overlap = np.intersect1d(X_df.columns.values, ch_impute_mask.columns.values)
ch_row_overlap = np.intersect1d(X_df.index.values, ch_impute_mask.index.values)
final_ch_impute_mask = ch_impute_mask.loc[ch_row_overlap, ch_col_overlap]
print(final_ch_impute_mask.shape)

cg_col_overlap = np.intersect1d(X_df.columns.values, cg_impute_mask.columns.values)
cg_row_overlap = np.intersect1d(X_df.index.values, cg_impute_mask.index.values)
final_cg_impute_mask = cg_impute_mask.loc[cg_row_overlap, cg_col_overlap]
print(final_cg_impute_mask.shape)

final_mask = pd.concat([final_ch_impute_mask, final_cg_impute_mask], axis=1)
final_mask = final_mask.reindex(X_df.index.values)
final_mask = final_mask.reindex(columns=X_df.columns.values)
final_mask = ~final_mask * 1
print(final_mask)

(11945, 257)
(11945, 94)
                                 GRIN2B_CG  CAMK2A_CG  AGPAT4_CG  TANC2_CG  \
Sample                                                                       
20210111-1863-preAb-PFC-B12_A3           1          1          1         1   
20210111-1863-preAb-PFC-B12_A23          1          0          1         0   
20210111-1863-preAb-PFC-B12_B10          1          0          0         1   
20210111-1863-preAb-PFC-B12_A1           1          1          1         1   
20210111-1863-preAb-PFC-B12_B13          0          0          1         1   
...                                    ...        ...        ...       ...   
20210224-5577-preAb-PFC-C04_O7           1          1          1         1   
20210224-5577-preAb-PFC-C04_L18          1          1          1         1   
20210224-5577-preAb-PFC-C04_P6           1          1          1         1   
20210224-5577-preAb-PFC-C04_P19          1          1          1         1   
20210224-5577-preAb-PFC-C04_O8         

In [ ]:
#data matrix, X
X=np.array(adata.X)
print(X.shape)

(11945, 351)


In [ ]:
class VAE:
  def __init__(self, input_shape,batch_size=256,optimizer=keras.optimizers.Adam(learning_rate=.001),epochs=50
               ,callback=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)):
    # self.mu=layers.Dense(32,name='mu')
    # self.sigma=layers.Dense(32,name='sigma')
    self.mu = None
    self.sigma = None

    self.input_shape=input_shape
    self.batch_size=batch_size
    self.n_epochs=epochs
    self.optimizer=optimizer
    # self.recon_loss_function=recon_loss_function
    self.callback=callback
    # self.model=self.generate_model_architecture()
    # self.model.compile(optimizer=optimizer,loss=self.vae_loss)
    self.encoder=self.generate_encoder()
    self.encoder.compile(optimizer=optimizer,loss=self.vae_loss)
    self.sampler=self.perform_sampling()
    # self.sampler.compile(optimizer=optimizer,loss=self.vae_loss)
    self.decoder=self.generate_decoder()
    self.decoder.compile(optimizer=optimizer,loss=self.vae_loss)
    self.kappa=1

  def vae_loss(self, y_true, y_pred):
    print(y_true, y_pred)
    recon = 0
    kl = K.mean(0.5*K.sum(K.exp(self.sigma) + K.square(self.mu) - 1. - self.sigma, axis=1))
    return 200*recon + self.kappa*kl

  def sample_z(self, args):
    vae_mu, vae_sigma = args
    eps = K.random_normal(shape=(K.shape(vae_mu)[0], K.shape(vae_mu)[1]), mean=0., stddev=1., seed=42)
    return vae_mu + K.exp(vae_sigma/2)*eps

  def kl_loss(self):
    return K.mean(0.5*K.sum(K.exp(self.sigma) + K.square(self.mu) - 1. - self.sigma, axis=1))

  def generate_encoder(self):
    mC_input=keras.Input(shape=(self.input_shape,),name='mC_input')
    dense1=layers.Dense(128,activation='relu', name='dense1')(mC_input)
    dense2=layers.Dense(64,activation='relu', name='dense2')(dense1)
    vae_mu=layers.Dense(32,activation='linear')(dense2)
    vae_sigma=layers.Dense(32,activation='linear')(dense2)

    return Model(mC_input, (vae_mu, vae_sigma), name='encoder')

  def perform_sampling(self):
    vae_mu = keras.Input(shape=(32,))
    vae_sigma = keras.Input(shape=(32,))
    out=layers.Lambda(self.sample_z)([vae_mu, vae_sigma])
    return Model([vae_mu, vae_sigma], out, name='sampler')

  def generate_decoder(self):
    input_latent = keras.Input(shape=(32,))
    out_pred=layers.Dense(self.input_shape,activation='linear',name='out_pred')(input_latent)
    return Model(input_latent, out_pred, name='decoder')

  """def generate_model_architecture(self):
    mC_input=keras.Input(shape=(self.input_shape,),name='mC_input')
    dense1=layers.Dense(128,activation='relu', name='dense1')(mC_input)
    dense2=layers.Dense(64,activation='relu', name='dense2')(dense1)
    self.mu=self.mu(dense2)
    self.sigma=self.sigma(dense2)
    latent=layers.Lambda(self.sample_z, output_shape=(32,), name='latent')([self.mu, self.sigma])
    # latent=layers.Dense(32,activation='relu', name='latent')(dense2)
    out_pred=layers.Dense(self.input_shape,activation='linear',name='out_pred')(latent)

    return Model(mC_input,out_pred)"""


  def train_vae(self,train_dataset,test_dataset):
    train_acc_metric = keras.metrics.MeanSquaredError()
    test_acc_metric = keras.metrics.MeanSquaredError()

    epochs = self.n_epochs
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        print(self.kappa)
        start_time = time.time()

        # Iterate over the batches of the dataset.
        for step, (x_batch_train) in enumerate(train_dataset):
            with tf.GradientTape() as encoder_tape, tf.GradientTape() as decoder_tape:
                self.mu, self.sigma =self.encoder(x_batch_train, training=True)
                latent = self.sampler([self.mu, self.sigma])
                logits=self.decoder(latent, training=True)
                # logits = self.model(x_batch_train, training=True)
                loss_value = self.vae_loss(x_batch_train, logits)
                # print(loss_value)
            encoder_grads = encoder_tape.gradient(loss_value, self.encoder.trainable_weights)
            decoder_grads = decoder_tape.gradient(loss_value, self.decoder.trainable_weights)
            self.optimizer.apply_gradients(zip(encoder_grads, self.encoder.trainable_weights))
            self.optimizer.apply_gradients(zip(decoder_grads, self.decoder.trainable_weights))
            # grads = tape.gradient(loss_value, self.model.trainable_weights)
            # self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))

            # Update training metric.
            train_acc_metric.update_state(x_batch_train, logits)

            # Log every 31 batches.
            if step % 31 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (step, float(loss_value))
                )
                #print("Seen so far: %d samples" % ((step + 1) * batch_size))

        # Display metrics at the end of each epoch.
        train_acc = train_acc_metric.result()
        print("Reconstruction Training acc over epoch: %.4f" % (float(train_acc),))
        print("KL Divergence: %.10f" % (float(self.kl_loss())))

        # Reset training metrics at the end of each epoch
        train_acc_metric.reset_states()

        # Run a validation loop at the end of each epoch.
        for x_batch_test in test_dataset:
            self.mu, self.sigma = self.encoder(x_batch_test, training=False)
            latent = self.sampler([self.mu, self.sigma])
            test_logits = self.decoder(latent, training=False)
            # test_logits = self.model(x_batch_test, training=False)
            # Update val metrics
            test_acc_metric.update_state(x_batch_test, test_logits)
        test_acc = test_acc_metric.result()
        test_acc_metric.reset_states()
        print("Validation acc: %.4f" % (float(test_acc),))
        #print("Time taken: %.2fs" % (time.time() - start_time))

        if self.kappa < 0.95:
            self.kappa += 0.25

  def predict(self,X):
    self.mu, self.sigma = self.encoder.predict(X)
    latent = self.sampler([self.mu, self.sigma])
    return self.decoder.predict(latent)

  def model_summary(self):
    print(self.model.summary())

  def visualize_model(self):
    #dunno how to make this show from the function lol
    keras.utils.plot_model(self.model, show_shapes=True, show_layer_names=True)

  def correlation_accuracy(self, real_data):
    reconstructed_data = self.predict(real_data)
    corrs = 0
    for i in range(len(reconstructed_data)):
        corrs += pearsonr(reconstructed_data[i], real_data[i])[0]

    print(corrs/len(reconstructed_data))

  def mse_error(self, real_data):
    reconstructed_data = self.predict(real_data)
    mse = 0
    for i in range(len(reconstructed_data)):
        mse += mean_squared_error(reconstructed_data[i], real_data[i])

    print(mse/len(reconstructed_data))




In [ ]:
# X_train, X_test = train_test_split(X, test_size=0.33, random_state=42,shuffle=True)
X_train, X_test = train_test_split(np.array(adata.X), test_size=0.33, random_state=42,shuffle=True)

In [ ]:
vae=VAE(X_train.shape[1])

In [ ]:
batch_size=256

train_dataset=tf.data.Dataset.from_tensor_slices(X_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_dataset = test_dataset.batch(batch_size)

In [ ]:
len(X_train)/batch_size

31.26171875

In [ ]:
vae.train_vae(train_dataset,test_dataset)


Start of epoch 0
1
tf.Tensor(
[[0.5140187  0.11111111 0.7804878  ... 0.6296296  0.04545455 0.463558  ]
 [0.757732   0.30769232 0.75       ... 0.7461929  0.09756097 0.18181819]
 [0.78488374 0.84745765 0.916      ... 0.87798035 0.7105263  0.9347826 ]
 ...
 [0.77397263 0.175      0.60655737 ... 0.5591398  0.08333334 0.6666667 ]
 [0.80246913 0.78431374 0.8632479  ... 0.70542634 0.75       0.11428571]
 [0.8906606  0.8239437  0.8101266  ... 0.80604136 0.5564516  0.47058824]], shape=(256, 351), dtype=float32)
tf.Tensor(
[[ 0.18615524  0.06209029 -0.177271   ... -0.2100995   0.13800792
   0.00820704]
 [-0.00245556 -0.53951865 -0.14125009 ...  0.18833135  0.11239978
  -0.5300702 ]
 [-0.10650368  0.00584707 -0.72578365 ... -0.3574647  -0.36140597
  -0.05015973]
 ...
 [ 0.27139008 -0.13174285 -1.2914472  ...  0.61817515 -0.28123257
  -0.14537776]
 [-0.7618611   0.92328095  0.24977316 ... -1.1995995   0.16648817
  -0.06972747]
 [ 0.28109065  0.3466923  -0.25281975 ... -0.30805027  0.6524317
  -0.

ValueError: ignored

In [ ]:
vae.predict(X_train)

In [ ]:
# vae.correlation_accuracy(X_train)
vae.correlation_accuracy(X_test)
# vae.mse_error(X_train)
vae.mse_error(X_test)

0.9663105203861073
0.008784314968768754
